In [2]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)
import catboost
from catboost import *
from catboost import datasets

(train_df, test_df) = catboost.datasets.amazon()

from sklearn.model_selection import train_test_split

y = train_df.ACTION
X = train_df.drop('ACTION', axis=1)

cat_features = list(range(0, X.shape[1]))

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

tunned_model = CatBoostClassifier(
    random_seed=63,                 # Фиксированный сид для воспроизводимости результатов
    iterations=1000,                 # Количество деревьев (итераций) в бустинге, увеличено для лучшего качества
    learning_rate=0.03,              # Скорость обучения, баланс между сходимостью и переобучением
    l2_leaf_reg=3,                   # Коэффициент L2-регуляризации на значения в листьях деревьев (предотвращает переобучение)
    bagging_temperature=1,           # Температура для случайности в бутстрапе (чем выше, тем более разнообразные деревья)
    random_strength=1,               # Величина шума для предотвращения переобучения в CTR-признаках
    one_hot_max_size=2,              # Порог для one-hot кодирования (больше — значит, больше признаков кодируются OHE)
    leaf_estimation_method='Newton'  # Метод обновления весов в листьях (Newton дольше но лучше качество может дать, чем Gradient)
)

tunned_model.fit(
    X_train, y_train,
    cat_features=cat_features,       # Список категориальных признаков
    verbose=False,                    # Отключение текстового вывода в консоль
    eval_set=(X_validation, y_validation),  # Валидационный набор для контроля переобучения
    plot=True                         # Включение графической визуализации обучения
)

best_model = CatBoostClassifier(
    random_seed=63,
    iterations=int(tunned_model.tree_count_ * 1.2),
)

best_model.fit(
    X, y,
    cat_features=cat_features,
    verbose=100
)

X_test = test_df.drop('id', axis=1)
test_pool = Pool(data=X_test, cat_features=cat_features)
contest_predictions = best_model.predict_proba(test_pool)
print('Predictoins:')
print(contest_predictions)

f = open('submit.csv', 'w')
f.write('Id,Action\n')
for idx in range(len(contest_predictions)):
    line = str(test_df['id'][idx]) + ',' + str(contest_predictions[idx][1]) + '\n'
    f.write(line)
f.close()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.042133
0:	learn: 0.6436102	total: 5.06ms	remaining: 5.52s
100:	learn: 0.1543140	total: 947ms	remaining: 9.3s
200:	learn: 0.1467631	total: 2.1s	remaining: 9.35s
300:	learn: 0.1430357	total: 3.27s	remaining: 8.59s
400:	learn: 0.1398797	total: 4.44s	remaining: 7.66s
500:	learn: 0.1371626	total: 5.63s	remaining: 6.66s
600:	learn: 0.1341771	total: 6.83s	remaining: 5.59s
700:	learn: 0.1310185	total: 8.03s	remaining: 4.49s
800:	learn: 0.1282023	total: 9.23s	remaining: 3.37s
900:	learn: 0.1255076	total: 10.4s	remaining: 2.22s
1000:	learn: 0.1231062	total: 11.6s	remaining: 1.07s
1092:	learn: 0.1209859	total: 12.8s	remaining: 0us
Predictoins:
[[0.3002 0.6998]
 [0.0116 0.9884]
 [0.012  0.988 ]
 ...
 [0.0053 0.9947]
 [0.0431 0.9569]
 [0.0125 0.9875]]
